In [7]:
import pandas as pd
import csv
import json
import requests
from datetime import datetime
import itertools
from collections import Counter

In [62]:
url = 'http://data.parlameter.si/v1/getVotesOfSessionTable/7654/'
data = pd.read_json(url)
coalition = requests.get('https://data.parlameter.si/v1/getCoalitionPGs').json()['coalition']
orgs = requests.get('https://data.parlameter.si/v1/getAllPGsExt/')
data['option_ni'] = 0
data['option_za'] = 0
data['option_proti'] = 0
data['option_kvorum'] = 0
data.loc[data['option'] == 'ni', 'option_ni'] = 1
data.loc[data['option'] == 'za', 'option_za'] = 1
data.loc[data['option'] == 'proti', 'option_proti'] = 1
data.loc[data['option'] == 'kvorum', 'option_kvorum'] = 1
data['voter_unit'] = 1
data['is_coalition'] = 0
data.loc[data['voterparty'].isin(coalition), 'is_coalition'] = 1

def getPercent(a, b, c, d=None):
    a = 0 if pd.isnull(a) else a
    b = 0 if pd.isnull(b) else b
    c = 0 if pd.isnull(c) else c
    if d:
        d = 0 if pd.isnull(d) else d
        devider = float(sum([a, b, c, d]))
        if devider:
            return max(a, b, c, d) / devider *100
        else:
            return 0
    else:
        devider = float(sum([a, b, c]))
        if devider:
            return max(a, b, c) / devider *100
        else:
            return 0

In [14]:
#za proti ni kvorum
all_votes = data.groupby('vote_id').sum()
#all_votes[['option_ni', 'option_za', 'option_proti', 'option_kvorum']]

In [17]:
all_votes['max_option_percent'] = all_votes.apply(lambda row: getOptions(row), axis=1)

In [18]:
all_votes[['option_za', 'option_kvorum', 'option_proti','option_ni', 'max_option_percent']]


,option_za,option_kvorum,option_proti,option_ni,max_option_percent
vote_id,,,,,
6668,55,1,23,11,"{'kvorum': 1, 'proti': 23, 'maxOptPerc': 69.62..."
6669,41,7,30,12,"{'kvorum': 7, 'proti': 30, 'maxOptPerc': 52.56..."
6670,53,5,20,12,"{'kvorum': 5, 'proti': 20, 'maxOptPerc': 67.94..."
6671,25,1,53,11,"{'kvorum': 1, 'proti': 53, 'maxOptPerc': 67.08..."
6672,25,1,50,14,"{'kvorum': 1, 'proti': 50, 'maxOptPerc': 65.78..."
6673,21,6,52,11,"{'kvorum': 6, 'proti': 52, 'maxOptPerc': 65.82..."
6674,52,2,24,12,"{'kvorum': 2, 'proti': 24, 'maxOptPerc': 66.66..."
6675,7,4,66,13,"{'kvorum': 4, 'proti': 66, 'maxOptPerc': 85.71..."
6678,18,12,49,11,"{'kvorum': 12, 'proti': 49, 'maxOptPerc': 62.0..."


In [5]:
#members kvorum
def getMPsList(row, proti):
    try:
        return str(list(proti[row.name].reset_index()['voter']))
    except:
        return []
    
def getPGsList(row, proti):
    try:
        return str(dict(Counter(list(proti[row.name].reset_index()['voterparty']))))
    except:
        return []

    #return row.name
m_proti = data[data.option_proti == 1].groupby(['vote_id']).apply(lambda x: x["voter"])
m_za = data[data.option_za == 1].groupby(['vote_id']).apply(lambda x: x["voter"])
m_ni = data[data.option_ni == 1].groupby(['vote_id']).apply(lambda x: x["voter"])
m_kvorum = data[data.option_kvorum == 1].groupby(['vote_id']).apply(lambda x: x["voter"])

pg_proti = data[data.option_proti == 1].groupby(['vote_id']).apply(lambda x: x["voterparty"])
pg_za = data[data.option_za == 1].groupby(['vote_id']).apply(lambda x: x["voterparty"])
pg_ni = data[data.option_ni == 1].groupby(['vote_id']).apply(lambda x: x["voterparty"])
pg_kvorum = data[data.option_kvorum == 1].groupby(['vote_id']).apply(lambda x: x["voterparty"])

all_votes['m_proti'] = all_votes.apply(lambda row: getMPsList(row, m_proti),axis=1)
all_votes['m_za'] = all_votes.apply(lambda row: getMPsList(row, m_za),axis=1)
all_votes['m_ni'] = all_votes.apply(lambda row: getMPsList(row, m_ni),axis=1)
all_votes['m_kvorum'] = all_votes.apply(lambda row: getMPsList(row, m_kvorum),axis=1)

all_votes['pg_proti'] = all_votes.apply(lambda row: getPGsList(row, pg_proti),axis=1)
all_votes['pg_za'] = all_votes.apply(lambda row: getPGsList(row, pg_za),axis=1)
all_votes['pg_ni'] = all_votes.apply(lambda row: getPGsList(row, pg_ni),axis=1)
all_votes['pg_kvorum'] = all_votes.apply(lambda row: getPGsList(row, pg_kvorum),axis=1)

#list(proti[6669].reset_index()['voter'])

In [74]:
def getOptions(row, side):
    maxOptionPercent = getPercent(row['option_za'],
                                  row['option_proti'],
                                  row['option_kvorum'],
                                  row['option_ni'])
    stats = {'for': row['option_za'],
             'against': row['option_proti'],
             'abstain': row['option_kvorum'],
             'not_present': row['option_ni']}
    max_opt = max(stats, key=stats.get)
    max_ids = [key for key, val in stats.iteritems() if val == max(stats.values())]
    if len(max_ids) > 1:
        if 'not_present' in max_ids:
            max_ids.remove('not_present')
            if len(max_ids) > 1:
                max_vote = 'cant_compute'
            else:
                max_vote = max_ids[0]
        else:
            max_vote = 'cant_compute'
    else:
        max_vote = max_ids[0]
    if side == 'oppo':
        outliers = [opt for opt in ['for', 'against'] if stats[opt]]
    else:
        outliers = [opt for opt in ['abstain', 'for', 'against'] if stats[opt]]
    for opt in max_ids:
        if opt in outliers:
            outliers.remove(opt)
    return json.dumps({'votes': {
                             'for': row['option_za'],
                             'against': row['option_proti'],
                             'abstain': row['option_kvorum'],
                             'not_present': row['option_ni'],
                             },
                       'max': {
                           'max_opt': max_vote,
                           'maxOptPerc': maxOptionPercent
                           },
                       'outliers': outliers})
all_votes['coal'] = data[data.is_coalition==1].groupby(['vote_id']).sum().apply(lambda row: getOptions(row, 'coal'), axis=1)
all_votes['oppo'] = data[data.is_coalition==0].groupby(['vote_id']).sum().apply(lambda row: getOptions(row, 'oppo'), axis=1)

In [75]:
#for i in data.voterparty.unique():
a = data.groupby(['vote_id', 'voterparty']).sum().apply(lambda row: getOptions(row, 'ps'), axis=1)

In [81]:
a[6668].keys()

Int64Index([1, 2, 3, 5, 6, 7, 8, 108, 109, 111], dtype='int64', name=u'voterparty')